In [ ]:
from google.colab import files


uploaded = files.upload()

Saving movies.csv to movies.csv


In [ ]:
import pandas as pd
import io

movies = pd.read_csv(io.BytesIO(uploaded['movies.csv']))
print(movies)

       movieId                                title  \
0            1                     Toy Story (1995)   
1            2                       Jumanji (1995)   
2            3              Grumpier Old Men (1995)   
3            4             Waiting to Exhale (1995)   
4            5   Father of the Bride Part II (1995)   
...        ...                                  ...   
10324   146684        Cosmic Scrat-tastrophe (2015)   
10325   146878           Le Grand Restaurant (1966)   
10326   148238       A Very Murray Christmas (2015)   
10327   148626                 The Big Short (2015)   
10328   149532  Marco Polo: One Hundred Eyes (2015)   

                                            genres  
0      Adventure|Animation|Children|Comedy|Fantasy  
1                       Adventure|Children|Fantasy  
2                                   Comedy|Romance  
3                             Comedy|Drama|Romance  
4                                           Comedy  
...                  

In [ ]:
from google.colab import files


uploaded = files.upload()

Saving ratings.csv to ratings.csv


In [ ]:
import pandas as pd
import io

ratings = pd.read_csv(io.BytesIO(uploaded['ratings.csv']))
print(ratings)

        userId  movieId  rating   timestamp
0            1       16     4.0  1217897793
1            1       24     1.5  1217895807
2            1       32     4.0  1217896246
3            1       47     4.0  1217896556
4            1       50     4.0  1217896523
...        ...      ...     ...         ...
105334     668   142488     4.0  1451535844
105335     668   142507     3.5  1451535889
105336     668   143385     4.0  1446388585
105337     668   144976     2.5  1448656898
105338     668   148626     4.5  1451148148

[105339 rows x 4 columns]


In [ ]:
# Merge datasets
movie_ratings = pd.merge(ratings, movies, on='movieId')

def popularity_based_recommender(movies, genre, min_reviews_threshold, num_recommendations):
    # Calculate movie statistics
    movie_stats = movie_ratings.groupby('movieId').agg({'rating': ['mean', 'count']})
    movie_stats.columns = ['Average Movie Rating', 'Num Reviews']

    # Merge movie statistics with movies dataset
    movies = pd.merge(movies, movie_stats, left_on='movieId', right_index=True)

    # Filter movies by genre
    genre_movies = movies[movies['genres'].str.contains(genre)]

    # Filter by minimum reviews threshold
    genre_movies = genre_movies[genre_movies['Num Reviews'] >= min_reviews_threshold]

    # Sort by popularity (descending order of average rating)
    genre_movies = genre_movies.sort_values(by=['Average Movie Rating', 'Num Reviews'], ascending=[False, False])

    # Select top N recommendations
    recommendations = genre_movies.head(num_recommendations)

    return recommendations

# Example usage
genre = 'Comedy'
min_reviews_threshold = 100
num_recommendations = 5
recommendations = popularity_based_recommender(movies, genre, min_reviews_threshold, num_recommendations)
print(recommendations[['movieId', 'title', 'Average Movie Rating', 'Num Reviews']])

     movieId                                   title  Average Movie Rating  \
922     1136  Monty Python and the Holy Grail (1975)              4.301948   
537      608                            Fargo (1996)              4.271144   
959     1197              Princess Bride, The (1987)              4.163743   
260      296                     Pulp Fiction (1994)              4.160000   
316      356                     Forrest Gump (1994)              4.138264   

     Num Reviews  
922          154  
537          201  
959          171  
260          325  
316          311  


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel



# Calculate TF-IDF matrix for genres
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['genres'])

# Calculate cosine similarity between movies based on genres
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

def content_based_recommender(movies, title, num_recommendations):
    # Get the index of the input movie
    idx = movies[movies['title'] == title].index[0]
    print(idx)

    # Get the similarity scores with all other movies
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the top N similar movies
    sim_indices = [x[0] for x in sim_scores[1:num_recommendations+1]]

    # Get the titles of the top N similar movies
    recommendations = movies.iloc[sim_indices]['title'].reset_index(drop=True)

    return recommendations

# Example usage
movie_title = 'Antz (1998)'
num_recommendations = 5
recommendations = content_based_recommender(movies, movie_title, num_recommendations)
print(recommendations)
print(recommendations[['movies', 'movie_title', 'num_recommendations']])


1815
0                                       Antz (1998)
1                                Toy Story 2 (1999)
2    Adventures of Rocky and Bullwinkle, The (2000)
3                  Emperor's New Groove, The (2000)
4                             Monsters, Inc. (2001)
Name: title, dtype: object


KeyError: "None of [Index(['movies', 'movie_title', 'num_recommendations'], dtype='object')] are in the [index]"

In [ ]:
def content_based_recommender(movie_title, N):
    # Find the genres of the given movie
    genres = movies[movies['title'] == movie_title]['genres'].values[0]

    # Find movies with similar genres
    similar_movies = movies[movies['genres'].str.contains(genres)]

    # Remove the given movie from the list
    similar_movies = similar_movies[similar_movies['title'] != movie_title]

    # Recommend top N movies
    top_N_movies = similar_movies.head(N)['title'].values
    return top_N_movies



In [ ]:
def collaborative_based_recommender(user_id, N, k):
    # Filter ratings for the given user
    user_ratings = ratings[ratings['userId'] == user_id]

    # Find movies rated by the given user
    user_movies = user_ratings['movieId'].values

    # Find similar users who have also rated these movies
    similar_users = ratings[ratings['movieId'].isin(user_movies)]
    similar_users = similar_users[similar_users['userId'] != user_id]

    # Calculate similarity between users based on ratings
    user_similarity = similar_users.groupby('userId').size().sort_values(ascending=False)
    similar_users = user_similarity.head(k).index.tolist()

    # Recommend top N movies based on these similar users' ratings
    recommended_movies = ratings[ratings['userId'].isin(similar_users)]
    recommended_movies = recommended_movies[~recommended_movies['movieId'].isin(user_movies)]
    recommended_movies = recommended_movies.groupby('movieId').size().sort_values(ascending=False)
    recommended_movies = recommended_movies.head(N)
    recommended_movies = movies[movies['movieId'].isin(recommended_movies.index)]['title'].values

    return recommended_movies


In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Create widgets for input
movie_input = widgets.Text(placeholder='Enter movie title')
user_input = widgets.IntText(placeholder='Enter user ID')
N_input = widgets.IntSlider(min=1, max=10, step=1, value=5, description='Num recommendations')
k_input = widgets.IntSlider(min=1, max=1000, step=1, value=100, description='Threshold for similar users')

# Create button for triggering recommendation
button = widgets.Button(description='Get Recommendations')

# Define function to handle button click
def on_button_click(b):
    movie_title = movie_input.value
    user_id = user_input.value
    N = N_input.value
    k = k_input.value

    if movie_title:
        recommendations = content_based_recommender(movie_title, N)
        print(f"Content-Based Recommendations for '{movie_title}':")
        for i, movie in enumerate(recommendations, 1):
            print(f"{i}. {movie}")
    elif user_id:
        recommendations = collaborative_based_recommender(user_id, N, k)
        print(f"Collaborative-Based Recommendations for User ID {user_id}:")
        for i, movie in enumerate(recommendations, 1):
            print(f"{i}. {movie}")

# Link button click event to function
button.on_click(on_button_click)

# Display widgets and button
display(movie_input, user_input, N_input, k_input, button)


Text(value='', placeholder='Enter movie title')

IntText(value=0)

IntSlider(value=5, description='Num recommendations', max=10, min=1)

IntSlider(value=100, description='Threshold for similar users', max=1000, min=1)

Button(description='Get Recommendations', style=ButtonStyle())

Content-Based Recommendations for 'Princess Bride, The (1987)':
1. Toy Story (1995)
2. Jumanji (1995)
3. Grumpier Old Men (1995)
4. Waiting to Exhale (1995)
5. Father of the Bride Part II (1995)
